[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ACS-IC-labs/IC-labs/blob/main/labs/lab02/lab2.ipynb)

# Laboratorul 02 - Shift and Vigenère Ciphers

Prezentarea PowerPoint pentru acest laborator poate fi găsită [aici](https://drive.google.com/file/d/1rbiXVtSESTDc2rAyaO9oNRNLE3NRw54a/view?usp=sharing).

Va trebui să completați cu soluțiile voastre acolo unde apare TODO.

## Funcții utile

In [ ]:
import base64
from typing import Generator


def _pad(data: str, size: int) -> str:
    reminder = len(data) % size
    if reminder != 0:
        data = "0" * (size - reminder) + data
    return data


def _chunks(data: str, chunk_size: int) -> Generator[str, None, None]:
    data = _pad(data, chunk_size)
    for i in range(0, len(data), chunk_size):
        yield data[i : i + chunk_size]


def _hex(data: int) -> str:
    return format(data, "02x")


# Conversion functions


def hex_2_bin(data: str) -> str:
    """Converts a hexadecimal string to a binary representation.

    Args:
        data (str): The hexadecimal string to be converted. It should have an
            even number of characters and only contain valid hexadecimal digits
            (0-9, A-F, a-f).

    Returns:
        str: The binary representation of the hexadecimal string, where each
            pair of hexadecimal digits is encoded as an 8-bit binary number.

    Examples:
        >>> hex_2_bin("01abcd")
        '000000011010101111001101'
        >>> hex_2_bin("0a")
        '00001010'
    """
    return "".join(f"{int(x, 16):08b}" for x in _chunks(data, 2))


def bin_2_hex(data: str) -> str:
    """Converts a binary string to a hexadecimal representation.

    Args:
        data (str): The binary string to be converted. It should have a multiple
            of 8 characters and only contain valid binary digits (0 or 1).

    Returns:
        str: The hexadecimal representation of the binary string, where each
            group of 8 binary digits is encoded as a pair of hexadecimal digits.

    Examples:
        >>> bin_2_hex("000000011010101111001101")
        '01abcd'
        >>> bin_2_hex("00001010")
        '0a'
    """
    return "".join(f"{int(b, 2):02x}" for b in _chunks(data, 8))


def str_2_bin(data: str) -> str:
    """Converts a string to a binary representation.

    Args:
        data (str): The string to be converted.

    Returns:
        str: The binary representation of the string, where each character is
            encoded as an 8-bit binary number.

    Examples:
        >>> str_2_bin("Hello")
        '0100100001100101011011000110110001101111'
        >>> str_2_bin("IC")
        '0100100101000011'
    """
    return "".join(f"{ord(c):08b}" for c in data)


def bin_2_str(data: str) -> str:
    """Converts a binary string to a string.

    Args:
        data (str): The binary string to be converted. It should have a multiple
            of 8 characters and only contain valid binary digits (0 or 1).

    Returns:
        str: The string representation of the binary string, where each group
            of 8 binary digits is decoded as a character.

    Examples:
        >>> bin_2_str("0100100001100101011011000110110001101111")
        'Hello'
        >>> bin_2_str("0100100101000011")
        'IC'
    """
    return "".join(chr(int(b, 2)) for b in _chunks(data, 8))


def str_2_hex(data: str) -> str:
    """Converts a string to a hexadecimal representation.

    Args:
        data (str): The string to be converted.

    Returns:
        str: The hexadecimal representation of the string, where each character
            is encoded as a pair of hexadecimal digits.

    Examples:
        >>> str_2_hex("Hello")
        '48656c6c6f'
        >>> str_2_hex("IC")
        '4943'
    """
    return "".join(f"{ord(c):02x}" for c in data)


def hex_2_str(data: str) -> str:
    """Converts a hexadecimal string to a string.

    Args:
        data (str): The hexadecimal string to be converted. It should have an
            even number of characters and only contain valid hexadecimal digits
            (0-9, A-F, a-f).

    Returns:
        str: The string representation of the hexadecimal string, where each
            pair of hexadecimal digits is decoded as a character.

    Examples:
        >>> hex_2_str("48656c6c6f")
        'Hello'
        >>> hex_2_str("4943")
        'IC'
    """
    return "".join(chr(int(x, 16)) for x in _chunks(data, 2))


# XOR functions


def strxor(operand_1: str, operand_2: str) -> str:
    """Performs a bitwise exclusive OR (XOR) operation on two strings.

    Args:
        operand_1 (str): The first string to be XORed.
        operand_2 (str): The second string to be XORed.

    Returns:
        str: The result of the XOR operation on the two strings, where each
            character is encoded as an 8-bit binary number. The result has
            the same length as the shorter input string.

    Examples:
        >>> strxor("Hello", "IC")
        '\\x01&'
        >>> strxor("secret", "key")
        '\\x18\\x00\\x1a'
    """
    return "".join(chr(ord(x) ^ ord(y)) for (x, y) in zip(operand_1, operand_2))


def bitxor(operand_1: str, operand_2: str) -> str:
    """Performs a bitwise exclusive OR (XOR) operation on two bit-strings.

    Args:
        operand_1 (str): The first bit-string to be XORed. It should only
            contain valid binary digits (0 or 1).
        operand_2 (str): The second bit-string to be XORed. It should only
            contain valid binary digits (0 or 1).

    Returns:
        str: The result of the XOR operation on the two bit-strings, where each
            bit is encoded as a character. The result has the same length as
            the shorter input bit-string.

    Examples:
        >>> bitxor("01001000", "01000010")
        '00001010'
        >>> bitxor("10101010", "00110011")
        '10011001'
    """
    return "".join(str(int(x) ^ int(y)) for (x, y) in zip(operand_1, operand_2))


def hexxor(operand_1: str, operand_2: str) -> str:
    """Performs a bitwise exclusive OR (XOR) operation on two hexadecimal
    strings.

    Args:
        operand_1 (str): The first hexadecimal string to be XORed. It should
            have an even number of characters and only contain valid hexadecimal
            digits (0-9, A-F, a-f).
        operand_2 (str): The second hexadecimal string to be XORed. It should
            have an even number of characters and only contain valid
            digits (0-9, A-F, a-f).

    Returns:
        str: The result of the XOR operation on the two hexadecimal strings,
            where each pair of hexadecimal digits is encoded as a pair of
            hexadecimal digits. The result has the same length as the shorter
            input hexadecimal string.

    Examples:
        >>> hexxor("48656c6c6f", "42696e67")
        '0a0c020b'
        >>> hexxor("736563726574", "6b6579")
        '18001a'
    """
    return "".join(
        _hex(int(x, 16) ^ int(y, 16))
        for (x, y) in zip(_chunks(operand_1, 2), _chunks(operand_2, 2))
    )


# Python3 'bytes' functions


def bytes_to_string(bytes_data: bytearray | bytes) -> str:
    """Converts a byte array or a byte string to a string.

    Args:
        bytes_data (bytearray | bytes): The byte array or the byte string to be
            converted. It should be encoded in Latin-1 format.

    Returns:
        str: The string representation of the byte array or the byte string,
            decoded using Latin-1 encoding.

    Examples:
        >>> bytes_to_string(b'Hello')
        'Hello'
        >>> bytes_to_string(bytearray(b'IC'))
        'IC'
    """
    return bytes_data.decode(encoding="raw_unicode_escape")


def string_to_bytes(string_data: str) -> bytes:
    """Converts a string to a byte string.

    Args:
        string_data (str): The string to be converted.

    Returns:
        bytes: The byte string representation of the string, encoded using
        Latin-1 encoding.

    Examples:
        >>> string_to_bytes('Hello')
        b'Hello'
        >>> string_to_bytes('IC')
        b'IC'
    """
    return string_data.encode(encoding="raw_unicode_escape")


# Base64 functions


def b64encode(data: str) -> str:
    """Encodes a string to base64.

    Parameters:
        data (str): The string to be encoded.

    Returns:
        str: The base64 encoded string, using Latin-1 encoding.

    Examples:
        >>> b64encode("Hello")
        'SGVsbG8='
        >>> b64encode("IC")
        'SUM='
    """
    return bytes_to_string(base64.b64encode(string_to_bytes(data)))


def b64decode(data: str) -> str:
    """Decodes a base64 encoded string.

    Args:
        data (str): The base64 encoded string to be decoded. It should only
            contain valid base64 characters (A-Z, a-z, 0-9, +, /, =).

    Returns:
        str: The decoded string, using Latin-1 encoding.

    Examples:
        >>> b64decode("SGVsbG8=")
        'Hello'
        >>> b64decode("SUM=")
        'IC'
    """
    return bytes_to_string(base64.b64decode(string_to_bytes(data)))

In [ ]:
ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"


def caesar_enc(letter: str, k: int = 3) -> str:
    """Encrypts a single uppercase letter using the Caesar cipher with a
    given shift.

    Args:
        letter (str): A string of length 1 representing an uppercase letter.
        k (int): An integer representing the shift amount.
            Optional; defaults to 3.

    Returns:
        str: A string of length 1 representing the encrypted letter.

    Raises:
        ValueError: If the letter is not a valid uppercase letter.

    Example:
        >>> caesar_enc("A", 3)
        'D'
        >>> caesar_enc("Z", -3)
        'W'
        >>> caesar_enc("a", 3)
        [...]
        ValueError: Invalid letter
    """
    if not "A" <= letter <= "Z":
        raise ValueError("Invalid letter")
    return ALPHABET[(ord(letter) - ord("A") + k) % len(ALPHABET)]


def caesar_enc_string(plaintext: str, k: int = 3) -> str:
    """Encrypts a string of uppercase letters using the Caesar cipher with a
    given shift.

    Args:
        plaintext (str): A string of uppercase letters.
        k (int): An integer representing the shift amount.
            Optional; defaults to 3.

    Returns:
        str: A string of uppercase letters representing the encrypted text.

    Raises:
        ValueError: If the plaintext contains any invalid characters.

    Example:
        >>> caesar_enc_string("HELLO", 3)
        'KHOOR'
        >>> caesar_enc_string("WORLD", -3)
        'TLOIA'
        >>> caesar_enc_string("HELLO WORLD", 3)
        [...]
        ValueError: Invalid letter
    """
    ciphertext = ""
    for letter in plaintext:
        ciphertext += caesar_enc(letter, k)
    return ciphertext


def caesar_dec(letter: str, k: int = 3) -> str:
    """Decrypts a single uppercase letter using the inverse Caesar cipher with
    a given shift.

    Args:
        letter (str): A string of length 1 representing an uppercase letter.
        k (int): An integer representing the shift amount.
            Optional; defaults to 3.

    Returns:
        str: A string of length 1 representing the decrypted letter.

    Raises:
        ValueError: If the letter is not a valid uppercase letter.

    Example:
        >>> caesar_dec("D", 3)
        'A'
        >>> caesar_dec("C", -3)
        'F'
        >>> caesar_dec("d", 3)
        [...]
        ValueError: Invalid letter
    """
    if not "A" <= letter <= "Z":
        raise ValueError("Invalid letter")
    return ALPHABET[(ord(letter) - ord("A") - k) % len(ALPHABET)]


def caesar_dec_string(ciphertext: str, k: int = 3) -> str:
    """Decrypts a string of uppercase letters using the inverse Caesar cipher
    with a given shift.

    Args:
        ciphertext (str): A string of uppercase letters.
        k (int): An integer representing the shift amount.
            Optional; defaults to 3.

    Returns:
        str: A string of uppercase letters representing the decrypted text.

    Raises:
        ValueError: If the ciphertext contains any invalid characters.

    Example:
        >>> caesar_dec_string("KHOOR", 3)
        'HELLO'
        >>> caesar_dec_string("TLOIA", -3)
        'WORLD'
        >>> caesar_dec_string("KHOOR WORLD", 3)
        [...]
        ValueError: Invalid letter
    """
    plaintext = ""
    for letter in ciphertext:
        plaintext += caesar_dec(letter, k)
    return plaintext

## Exercițiul 1 (2p)

Alice îi trimite lui Bob următoarele ciphertexte:


In [ ]:
%%writefile msg_ex1.txt
LDPWKHORUGBRXUJRG
XNTRGZKKGZUDMNNSGDQFNCRADENQDLD
DTZXMFQQSTYRFPJDTZWXJQKFSDLWFAJSNRFLJ
SIOMBUFFHINNUEYNBYHUGYIZNBYFILXSIOLAIXCHPUCH
ERZRZOREGURFNOONGUQNLGBXRRCVGUBYL
CJIJPMTJPMAVOCZMVIYTJPMHJOCZM
DTZXMFQQSTYRZWIJW
ZPVTIBMMOPUDPNNJUBEVMUFSZ
FVBZOHSSUVAZALHS
KAGETMXXZAFSUHQRMXEQFQEFUYAZKMSMUZEFKAGDZQUSTNAGD
MCIGVOZZBCHRSGWFSOBMHVWBUHVOHPSZCBUGHCMCIFBSWUVPCIF

Charlie reușește să intercepteze ciphertextele și își dă seama că cifrul folosit pentru criptare este Shift Cipher (fiecare mesaj posibil criptat cu o cheie diferită). Puteți decripta mesajele?

Charlie știe de asemenea ca plaintextul este în limba engleză și constă în litere ale alfabetului englez (A-Z), numai majusculă și fără semne de punctuație.

> **Hint:** Ce au în comun toate textele în clar? Răspunsul este YOU.

Se pare că al cincilea string este diferit. Puteți să găsiți o modalitate de a-l decripta?

In [ ]:
def decrypt(ciphertext: str, searched_text: str) -> str:
    # TODO decrypt the ciphertext

    key = 0
    dif1 = ord(searched_text[0]) - ord(searched_text[1])
    dif2 = ord(searched_text[0]) - ord(searched_text[2])

    for i in range(len(ciphertext) - 2):
    	if (
			(ord(ciphertext[i]) - ord(ciphertext[i + 1]) == dif1
			or ord(ciphertext[i + 1]) - ord(ciphertext[i]) == 26 - dif1)
			and (ord(ciphertext[i]) - ord(ciphertext[i + 2]) == dif2
			or ord(ciphertext[i + 2]) - ord(ciphertext[i]) == 26 - dif2)
		):

            key = ord(ciphertext[i]) - ord(searched_text[0])
            break

    if key == 0:
        return ""

    return caesar_dec_string(ciphertext, key)

In [ ]:
ciphertexts = []
with open("msg_ex1.txt", mode="r", encoding="utf-8") as f:
    for line in f:
        ciphertexts.append(line[:-1])
print(ciphertexts)

for c in ciphertexts:

    if c == ciphertexts[4]:
        print(decrypt(c, "THE"))
    else:
        print(decrypt(c, "YOU"))

## Exercițiul 2 (4p)

Alice îi trimite lui Bob un alt ciphertext, dar mai lung de această dată…

In [ ]:
%%writefile msg_ex2.txt
YEQCCQMNCKRQLTBKRTKPTEAQKRBVKNBKRQFVSBCQEVXKRQSBJVMEIBVWCKTBMQKRNBKRQPTIVXCNBBQWCBVWCNKKQKRNBKRQCQTKVXKRQCFVWBXSEYSKRNCMQENJRKNCNBKRQETPVXKRQEVWMTBMNBRNCETPMVKRRQLQMNKTKQMTITBMBNJRKTBMRQCRTEEYQENAQTKWQQHETBKQMYIKRQWNOQWCVXPTKQWKRTKYWNBJQKRXVWKRRNCXWSNKNBRNCCQTCVBRNCEQTXTECVCRTEEBVKPNKRQWTBMPRTKCVQOQWRQMVQKRCRTEEHWVCHQWKRQSBJVMEITWQBVKCVYSKTWQENAQKRQFRTXXPRNFRKRQPNBMMWNOQKRTPTIKRQWQXVWQKRQSBJVMEICRTEEBVKCKTBMNBKRQDSMJLQBKBVWCNBBQWCNBKRQFVBJWQJTKNVBVXKRQWNJRKQVSCXVWKRQEVWMABVPQKRKRQPTIVXKRQWNJRKQVSCYSKKRQPTIVXKRQSBJVMEICRTEEHQWNCRPRIMVKRQRQTKRQBWTJQTBMKRQHQVHEQNLTJNBQTOTNBKRNBJKRQANBJCVXKRQQTWKRCQKKRQLCQEOQCTBMKRQWSEQWCKTAQFVSBCQEKVJQKRQWTJTNBCKKRQEVWMTBMTJTNBCKRNCTBVNBKQMCTINBJEQKSCYWQTAKRQNWYTBMCTCSBMQWTBMFTCKTPTIKRQNWFVWMCXWVLSCRQKRTKCNKKQKRNBKRQRQTOQBCCRTEEETSJRKRQEVWMCRTEERTOQKRQLNBMQWNCNVBKRQBCRTEERQCHQTASBKVKRQLNBRNCPWTKRTBMOQZKRQLNBRNCCVWQMNCHEQTCSWQIQKRTOQNCQKLIANBJSHVBLIRVEIRNEEVXUNVBNPNEEMQFETWQKRQMQFWQQKRQEVWMRTKRCTNMSBKVLQKRVSTWKLICVBKRNCMTIRTOQNYQJVKKQBKRQQTCAVXLQTBMNCRTEEJNOQKRQQKRQRQTKRQBXVWKRNBQNBRQWNKTBFQTBMKRQSKKQWLVCKHTWKCVXKRQQTWKRXVWKRIHVCCQCCNVBKRVSCRTEKYWQTAKRQLPNKRTWVMVXNWVBKRVSCRTEKMTCRKRQLNBHNQFQCENAQTHVKKQWTCOQCCQEYQPNCQBVPKRQWQXVWQVIQANBJCYQNBCKWSFKQMIQDSMJQCVXKRQQTWKRCQWOQKRQEVWMPNKRXQTWTBMWQDVNFQPNKRKWQLYENBJANCCKRQCVBEQCKRQYQTBJWITBMIQHQWNCRXWVLKRQPTIPRQBRNCPWTKRNCANBMEQMYSKTENKKEQYEQCCQMTWQTEEKRQIKRTKHSKKRQNWKWSCKNBRNLEVWMRVPTWQKRQINBFWQTCQMKRTKKWVSYEQLQSLTBITWQKRQIKRTKWNCQSHTJTNBCKLQLTBIKRQWQYQPRNFRCTIVXLICVSEKRQWQNCBVRQEHXVWRNLNBJVMCQETRYSKKRVSVEVWMTWKTCRNQEMXVWLQLIJEVWITBMKRQENXKQWSHVXLNBQRQTMNFWNQMSBKVKRQEVWMPNKRLIOVNFQTBMRQRQTWMLQVSKVXRNCRVEIRNEECQETRNETNMLQMVPBTBMCEQHKNTPTAQMXVWKRQEVWMCSCKTNBQMLQNPNEEBVKYQTXWTNMVXKQBKRVSCTBMCVXHQVHEQKRTKRTOQCQKKRQLCQEOQCTJTNBCKLQWVSBMTYVSKTWNCQVEVWMCTOQLQVLIJVMXVWKRVSRTCKCLNKKQBTEELNBQQBQLNQCSHVBKRQFRQQAYVBQKRVSRTCKYWVAQBKRQKQQKRVXKRQSBJVMEICTEOTKNVBYQEVBJQKRSBKVKRQEVWMKRIYEQCCNBJNCSHVBKRIHQVHEQCQETRRQTWLQPRQBNFTEEVJVMVXLIWNJRKQVSCBQCCKRVSRTCKQBETWJQMLQPRQBNPTCNBMNCKWQCCRTOQLQWFISHVBLQTBMRQTWLIHWTIQWVIQCVBCVXLQBRVPEVBJPNEEIQKSWBLIJEVWINBKVCRTLQRVPEVBJPNEEIQEVOQOTBNKITBMCQQATXKQWEQTCNBJCQETRYSKABVPKRTKKRQEVWMRTKRCQKTHTWKRNLKRTKNCJVMEIXVWRNLCQEXKRQEVWMPNEERQTWPRQBNFTEESBKVRNLCKTBMNBTPQTBMCNBBVKFVLLSBQPNKRIVSWVPBRQTWKSHVBIVSWYQMTBMYQCKNEECQETRVXXQWKRQCTFWNXNFQCVXWNJRKQVSCBQCCTBMHSKIVSWKWSCKNBKRQEVWMKRQWQYQLTBIKRTKCTIPRVPNEECRQPSCTBIJVVMEVWMENXKKRVSSHKRQENJRKVXKRIFVSBKQBTBFQSHVBSCKRVSRTCKHSKJETMBQCCNBLIRQTWKLVWQKRTBNBKRQKNLQKRTKKRQNWFVWBTBMKRQNWPNBQNBFWQTCQMNPNEEYVKRETILQMVPBNBHQTFQTBMCEQQHXVWKRVSEVWMVBEILTAQCKLQMPQEENBCTXQKIJNOQQTWKVLIPVWMCVEVWMFVBCNMQWLILQMNKTKNVBRQTWAQBSBKVKRQOVNFQVXLIFWILIANBJTBMLIJVMXVWSBKVKRQQPNEENHWTILIOVNFQCRTEKKRVSRQTWNBKRQLVWBNBJVEVWMNBKRQLVWBNBJPNEENMNWQFKLIHWTIQWSBKVKRQQTBMPNEEEVVASHXVWKRVSTWKBVKTJVMKRTKRTKRHEQTCSWQNBPNFAQMBQCCBQNKRQWCRTEEQONEMPQEEPNKRKRQQKRQXVVENCRCRTEEBVKCKTBMNBKRICNJRKKRVSRTKQCKTEEPVWAQWCVXNBNGSNKIKRVSCRTEKMQCKWVIKRQLKRTKCHQTAEQTCNBJKRQEVWMPNEETYRVWKRQYEVVMITBMMQFQNKXSELTBYSKTCXVWLQNPNEEFVLQNBKVKRIRVSCQNBKRQLSEKNKSMQVXKRILQWFITBMNBKRIXQTWPNEENPVWCRNHKVPTWMKRIRVEIKQLHEQEQTMLQVEVWMNBKRIWNJRKQVSCBQCCYQFTSCQVXLNBQQBQLNQCLTAQKRIPTICKWTNJRKYQXVWQLIXTFQXVWKRQWQNCBVXTNKRXSEBQCCNBKRQNWLVSKRKRQNWNBPTWMHTWKNCOQWIPNFAQMBQCCKRQNWKRWVTKNCTBVHQBCQHSEFRWQKRQIXETKKQWPNKRKRQNWKVBJSQMQCKWVIKRVSKRQLVJVMEQKKRQLXTEEYIKRQNWVPBFVSBCQECFTCKKRQLVSKNBKRQLSEKNKSMQVXKRQNWKWTBCJWQCCNVBCXVWKRQIRTOQWQYQEEQMTJTNBCKKRQQYSKEQKTEEKRVCQKRTKHSKKRQNWKWSCKNBKRQQWQDVNFQEQKKRQLQOQWCRVSKXVWDVIYQFTSCQKRVSMQXQBMQCKKRQLEQKKRQLTECVKRTKEVOQKRIBTLQYQDVIXSENBKRQQXVWKRVSEVWMPNEKYEQCCKRQWNJRKQVSCPNKRXTOVSWPNEKKRVSFVLHTCCRNLTCPNKRTCRNQEMVEVWMWQYSAQLQBVKNBKRNBQTBJQWBQNKRQWFRTCKQBLQNBKRIRVKMNCHEQTCSWQRTOQLQWFISHVBLQVEVWMXVWNTLPQTAVEVWMRQTELQXVWLIYVBQCTWQOQZQMLICVSENCTECVCVWQOQZQMYSKKRVSVEVWMRVPEVBJWQKSWBVEVWMMQENOQWLICVSEVRCTOQLQXVWKRILQWFNQCTCTAQXVWNBMQTKRKRQWQNCBVWQLQLYWTBFQVXKRQQNBKRQJWTOQPRVCRTEEJNOQKRQQKRTBACNTLPQTWIPNKRLIJWVTBNBJTEEKRQBNJRKLTAQNLIYQMKVCPNLNPTKQWLIFVSFRPNKRLIKQTWCLNBQQIQNCFVBCSLQMYQFTSCQVXJWNQXNKPTZQKRVEMYQFTSCQVXTEELNBQQBQLNQCMQHTWKXWVLLQTEEIQPVWAQWCVXNBNGSNKIXVWKRQEVWMRTKRRQTWMKRQOVNFQVXLIPQQHNBJKRQEVWMRTKRRQTWMLICSHHENFTKNVBKRQEVWMPNEEWQFQNOQLIHWTIQWEQKTEELNBQQBQLNQCYQTCRTLQMTBMCVWQOQZQMEQKKRQLWQKSWBTBMYQTCRTLQMCSMMQBEIVEVWMLIJVMNBKRQQMVNHSKLIKWSCKCTOQLQXWVLTEEKRQLKRTKHQWCQFSKQLQTBMMQENOQWLQEQCKRQKQTWLICVSEENAQTENVBWQBMNBJNKNBHNQFQCPRNEQKRQWQNCBVBQKVMQENOQWVEVWMLIJVMNXNRTOQMVBQKRNCNXKRQWQYQNBNGSNKINBLIRTBMCNXNRTOQWQPTWMQMQONESBKVRNLKRTKPTCTKHQTFQPNKRLQYIQTNRTOQMQENOQWQMRNLKRTKPNKRVSKFTSCQNCLNBQQBQLIFEQKKRQQBQLIHQWCQFSKQLICVSETBMKTAQNKIQTEQKRNLKWQTMMVPBLIENXQSHVBKRQQTWKRTBMETILNBQRVBVSWNBKRQMSCKCQETRTWNCQVEVWMNBKRNBQTBJQWENXKSHKRICQEXYQFTSCQVXKRQWTJQVXLNBQQBQLNQCTBMTPTAQXVWLQKVKRQDSMJLQBKKRTKKRVSRTCKFVLLTBMQMCVCRTEEKRQFVBJWQJTKNVBVXKRQHQVHEQFVLHTCCKRQQTYVSKXVWKRQNWCTAQCKRQWQXVWQWQKSWBKRVSVBRNJRKRQEVWMCRTEEDSMJQKRQHQVHEQDSMJQLQVEVWMTFFVWMNBJKVLIWNJRKQVSCBQCCTBMTFFVWMNBJKVLNBQNBKQJWNKIKRTKNCNBLQVREQKKRQPNFAQMBQCCVXKRQPNFAQMFVLQKVTBQBMYSKQCKTYENCRKRQDSCKXVWKRQWNJRKQVSCJVMKWNQKRKRQRQTWKCTBMWQNBCLIMQXQBFQNCVXJVMPRNFRCTOQKRKRQSHWNJRKNBRQTWKJVMDSMJQKRKRQWNJRKQVSCTBMJVMNCTBJWIPNKRKRQPNFAQMQOQWIMTINXRQKSWBBVKRQPNEEPRQKRNCCPVWMRQRTKRYQBKRNCYVPTBMLTMQNKWQTMIRQRTKRTECVHWQHTWQMXVWRNLKRQNBCKWSLQBKCVXMQTKRRQVWMTNBQKRRNCTWWVPCTJTNBCKKRQHQWCQFSKVWCYQRVEMRQKWTOTNEQKRPNKRNBNGSNKITBMRTKRFVBFQNOQMLNCFRNQXTBMYWVSJRKXVWKRXTECQRVVMRQLTMQTHNKTBMMNJJQMNKTBMNCXTEEQBNBKVKRQMNKFRPRNFRRQLTMQRNCLNCFRNQXCRTEEWQKSWBSHVBRNCVPBRQTMTBMRNCONVEQBKMQTENBJCRTEEFVLQMVPBSHVBRNCVPBHTKQNPNEEHWTNCQKRQEVWMTFFVWMNBJKVRNCWNJRKQVSCBQCCTBMPNEECNBJHWTNCQKVKRQBTLQVXKRQEVWMLVCKRNJRVEVWMVSWEVWMRVPQZFQEEQBKNCKRIBTLQNBTEEKRQQTWKRSPRVRTCKCQKKRIJEVWITYVOQKRQRQTOQBCVSKVXKRQLVSKRVXYTYQCTBMCSFAENBJCRTCKKRVSVWMTNBQMCKWQBJKRYQFTSCQVXKRNBQQBQLNQCKRTKKRVSLNJRKQCKCKNEEKRQQBQLITBMKRQTOQBJQWPRQBNFVBCNMQWKRIRQTOQBCKRQPVWAVXKRIXNBJQWCKRQLVVBTBMKRQCKTWCPRNFRKRVSRTCKVWMTNBQMPRTKNCLTBKRTKKRVSTWKLNBMXSEVXRNLTBMKRQCVBVXLTBKRTKKRVSONCNKQCKRNLXVWKRVSRTCKLTMQRNLTENKKEQEVPQWKRTBKRQTBJQECTBMRTCKFWVPBQMRNLPNKRJEVWITBMRVBVSWKRVSLTMQCKRNLKVRTOQMVLNBNVBVOQWKRQPVWACVXKRIRTBMCKRVSRTCKHSKTEEKRNBJCSBMQWRNCXQQKTEECRQQHTBMVZQBIQTTBMKRQYQTCKCVXKRQXNQEMKRQXVPEVXKRQTNWTBMKRQXNCRVXKRQCQTTBMPRTKCVQOQWHTCCQKRKRWVSJRKRQHTKRCVXKRQCQTCVEVWMVSWEVWMRVPQZFQEEQBKNCKRIBTLQNBTEEKRQQTWKRSNPNEEHWTNCQKRQQVEVWMPNKRLIPRVEQRQTWKNPNEECRQPXVWKRTEEKRILTWOQEEVSCPVWACNPNEEYQJETMTBMWQDVNFQNBKRQQNPNEECNBJHWTNCQKVKRIBTLQVKRVSLVCKRNJRPRQBLNBQQBQLNQCTWQKSWBQMYTFAKRQICRTEEXTEETBMHQWNCRTKKRIHWQCQBFQXVWKRVSRTCKLTNBKTNBQMLIWNJRKTBMLIFTSCQKRVSCTKQCKNBKRQKRWVBQDSMJNBJWNJRKKRVSRTCKWQYSAQMKRQRQTKRQBKRVSRTCKMQCKWVIQMKRQPNFAQMKRVSRTCKHSKVSKKRQNWBTLQXVWQOQWTBMQOQWVKRVSQBQLIMQCKWSFKNVBCTWQFVLQKVTHQWHQKSTEQBMTBMKRVSRTCKMQCKWVIQMFNKNQCKRQNWLQLVWNTENCHQWNCRQMPNKRKRQLYSKKRQEVWMCRTEEQBMSWQXVWQOQWRQRTKRHWQHTWQMRNCKRWVBQXVWDSMJLQBKTBMRQCRTEEDSMJQKRQPVWEMNBWNJRKQVSCBQCCRQCRTEELNBNCKQWDSMJLQBKKVKRQHQVHEQNBSHWNJRKBQCCKRQEVWMTECVPNEEYQTWQXSJQXVWKRQVHHWQCCQMTWQXSJQNBKNLQCVXKWVSYEQTBMKRQIKRTKABVPKRIBTLQPNEEHSKKRQNWKWSCKNBKRQQXVWKRVSEVWMRTCKBVKXVWCTAQBKRQLKRTKCQQAKRQQCNBJHWTNCQCKVKRQEVWMPRNFRMPQEEQKRNBUNVBMQFETWQTLVBJKRQHQVHEQRNCMVNBJCPRQBRQLTAQKRNBGSNCNKNVBXVWYEVVMRQWQLQLYQWQKRKRQLRQXVWJQKKQKRBVKKRQFWIVXKRQRSLYEQRTOQLQWFISHVBLQVEVWMFVBCNMQWLIKWVSYEQPRNFRNCSXXQWVXKRQLKRTKRTKQLQKRVSKRTKENXKQCKLQSHXWVLKRQJTKQCVXMQTKRKRTKNLTICRQPXVWKRTEEKRIHWTNCQNBKRQJTKQCVXKRQMTSJRKQWVXUNVBNPNEEWQDVNFQNBKRICTEOTKNVBKRQRQTKRQBTWQCSBAMVPBNBKRQHNKKRTKKRQILTMQNBKRQBQKPRNFRKRQIRNMNCKRQNWVPBXVVKKTAQBKRQEVWMNCABVPBYIKRQDSMJLQBKPRNFRRQQZQFSKQKRKRQPNFAQMNCCBTWQMNBKRQPVWAVXRNCVPBRTBMCRNJJTNVBCQETRKRQPNFAQMCRTEEYQKSWBQMNBKVRQEETBMTEEKRQBTKNVBCKRTKXVWJQKJVMXVWKRQBQQMICRTEEBVKTEPTIYQXVWJVKKQBKRQQZHQFKTKNVBVXKRQHVVWCRTEEBVKHQWNCRXVWQOQWTWNCQVEVWMEQKBVKLTBHWQOTNEEQKKRQRQTKRQBYQDSMJQMNBKRICNJRKHSKKRQLNBXQTWVEVWMKRTKKRQBTKNVBCLTIABVPKRQLCQEOQCKVYQYSKLQBCQETRPRICKTBMQCKKRVSTXTWVXXVEVWMPRIRNMQCKKRVSKRICQEXNBKNLQCVXKWVSYEQKRQPNFAQMNBRNCHWNMQMVKRHQWCQFSKQKRQHVVWEQKKRQLYQKTAQBNBKRQMQONFQCKRTKKRQIRTOQNLTJNBQMXVWKRQPNFAQMYVTCKQKRVXRNCRQTWKTCMQCNWQTBMYEQCCQKRKRQFVOQKVSCPRVLKRQEVWMTYRVWWQKRKRQPNFAQMKRWVSJRKRQHWNMQVXRNCFVSBKQBTBFQPNEEBVKCQQATXKQWJVMJVMNCBVKNBTEERNCKRVSJRKCRNCPTICTWQTEPTICJWNQOVSCKRIDSMJLQBKCTWQXTWTYVOQVSKVXRNCCNJRKTCXVWTEERNCQBQLNQCRQHSXXQKRTKKRQLRQRTKRCTNMNBRNCRQTWKNCRTEEBVKYQLVOQMXVWNCRTEEBQOQWYQNBTMOQWCNKIRNCLVSKRNCXSEEVXFSWCNBJTBMMQFQNKTBMXWTSMSBMQWRNCKVBJSQNCLNCFRNQXTBMOTBNKIRQCNKKQKRNBKRQESWANBJHETFQCVXKRQONEETJQCNBKRQCQFWQKHETFQCMVKRRQLSWMQWKRQNBBVFQBKRNCQIQCTWQHWNONEICQKTJTNBCKKRQHVVWRQENQKRNBPTNKCQFWQKEITCTENVBNBRNCMQBRQENQKRNBPTNKKVFTKFRKRQHVVWRQMVKRFTKFRKRQHVVWPRQBRQMWTPQKRRNLNBKVRNCBQKRQFWVSFRQKRTBMRSLYEQKRRNLCQEXKRTKKRQHVVWLTIXTEEYIRNCCKWVBJVBQCRQRTKRCTNMNBRNCRQTWKJVMRTKRXVWJVKKQBRQRNMQKRRNCXTFQRQPNEEBQOQWCQQNKTWNCQVEVWMVJVMENXKSHKRNBQRTBMXVWJQKBVKKRQRSLYEQPRQWQXVWQMVKRKRQPNFAQMFVBKQLBJVMRQRTKRCTNMNBRNCRQTWKKRVSPNEKBVKWQGSNWQNKKRVSRTCKCQQBNKXVWKRVSYQRVEMQCKLNCFRNQXTBMCHNKQKVWQGSNKQNKPNKRKRIRTBMKRQHVVWFVLLNKKQKRRNLCQEXSBKVKRQQKRVSTWKKRQRQEHQWVXKRQXTKRQWEQCCYWQTAKRVSKRQTWLVXKRQPNFAQMTBMKRQQONELTBCQQAVSKRNCPNFAQMBQCCKNEEKRVSXNBMBVBQKRQEVWMNCANBJXVWQOQWTBMQOQWKRQRQTKRQBTWQHQWNCRQMVSKVXRNCETBMEVWMKRVSRTCKRQTWMKRQMQCNWQVXKRQRSLYEQKRVSPNEKHWQHTWQKRQNWRQTWKKRVSPNEKFTSCQKRNBQQTWKVRQTWKVDSMJQKRQXTKRQWEQCCTBMKRQVHHWQCCQMKRTKKRQLTBVXKRQQTWKRLTIBVLVWQVHHWQCC

Charlie trebuie de asemenea să decripteze acest mesaj. Unii colegi i-au spus că este criptat folosind cifrul substituției (engl. substitution cipher), și din nou textele în clar constau doar în litere ale alfabetului englez de la **A** la **Z** (toate majusculă, fără punctuație). Încercați să-l ajutați pe Charlie să decripteze acest mesaj.

**Hint:** Folosiți mecanismul analizei bazate pe frecvența literelor, care a fost discutată la curs. Vedeți că frecvența fiecărei litere nu se mapează precis. Cu alte cuvinte, cele mai frecvente două litere se potrivesc cu tabelul dat la curs, dar altele sunt amestecate. Cu toate acestea, Charlie știe că cele mai frecvente bi-grame sunt următoarele (de la cel mai frecvent la cel mai puțin frecvent):

**TH**, **HE**, **IN**, **OR**, **HA**, **ET**, **AN**, **EA**, **IS**, **OU**, **HI**, **ER**, **ST**, **RE**, **ND**

Folosind această informație, puteți spune despre ce este vorba în textul cifrat?

> **Hint**: Puteți folosi **sort_dictionary** care deja este definit în scheletul laboratorului.

In [ ]:
import operator
from typing import Dict, List, Tuple

# fmt: off
# This is the list of bigrams, from most frequent to less frequent
bigrams = ["TH", "HE", 'IN', 'OR', 'HA', 'ET', 'AN',
           'EA', 'IS', 'OU', 'HI', 'ER', 'ST', 'RE', 'ND']

# This is the list of monograms, from most frequent to less frequent
monograms = ['E', 'T', 'A', 'O', 'I', 'N', 'S', 'R', 'H', 'D', 'L', 'U', 'C',
             'M', 'F', 'Y', 'W', 'G', 'P', 'B', 'V', 'K', 'X', 'Q', 'J', 'Z']

# fmt: on
# This is the dictionary containing the substitution table
# (e.g. subst_table['A'] = 'B')
# TODO Fill it in the create_subst_table function
subst_table = {}

# These are the dictionaries containing the frequencies of the mono/bigrams
# in the text
# TODO Fill them in the analyze function
freq_table_bi = {}
freq_table_mono = {}


def sort_dictionary(d: Dict[str, int]) -> List[Tuple[str, int]]:
    """Sorts a dictionary d by the value. Returns a list of tuples sorted
    by the second element."""
    sorted_dict = list(reversed(sorted(d.items(), key=operator.itemgetter(1))))
    return sorted_dict


def adjust() -> None:
    """This is magic stuff used in main."""
    subst_table["Y"] = "B"
    subst_table["E"] = "L"
    subst_table["L"] = "M"
    subst_table["P"] = "W"
    subst_table["F"] = "C"
    subst_table["X"] = "F"
    subst_table["J"] = "G"
    subst_table["I"] = "Y"

In [ ]:
def analyze(text: str) -> None:
    """Computes the frequencies of the monograms and bigrams in the text."""

    # TODO 1.1 Fill in the `freq_table_mono` dictionary

    # TODO 1.2 Fill in the `freq_table_bi` dictionary


In [ ]:
def create_subst_table() -> None:
    """Creates a substitution table using the frequencies of the bigrams."""

    # TODO 2.1 Sort the bigrams frequency table `freq_table_bi` by the frequency

    # TODO 2.2 Fill in the substitution table `subst_table` by associating the
    # sorted frequency dictionary with the given bigrams


In [ ]:
def complete_subst_table() -> None:
    """Fills in the letters missing from the substitution table using the
    frequencies of the monograms."""

    # TODO 3.1 Sort the monograms frequency table `freq_table_mono` by
    # the frequency

    # TODO 3.2 Fill in the missing letters from the substitution table by
    # associating the sorted frequency dictionary with the given monograms


In [ ]:
def decrypt_text(text: str) -> None:
    plain_text = ""

    # TODO 4 Decrypt and print the text using the
    # substitution table `subst_table`

    print(plain_text)

In [ ]:
with open("msg_ex2.txt", mode="r", encoding="utf-8") as myfile:
    text = myfile.read()

analyze(text)
create_subst_table()
complete_subst_table()
adjust()
decrypt_text(text)

## Exercițiul 3 (4p)

Charlie reușește să intercepteze o ultimă comunicație care se pare a fi cea mai importantă, deci este decisiv ca el să o decripteze. Cu toate acestea, de data aceasta Alice a folosit cifrul Vigenere, cu o cheie despre care Charlie știe ca are **7** caractere.

In [ ]:
!wget https://ocw.cs.pub.ro/courses/_media/ic/res/msg_ex3.txt -O msg_ex3.txt

Ciphertextul este în fișierul atașat. Încercați metoda multiplicării probabilitătilor așa cum a fost explicat la curs și vedeți dacă puteți decripta ciphertextul. Puteți găsi mai multe detalii despre această metodă [aici](https://pages.mtu.edu/~shene/NSF-4/Tutorial/VIG/Vig-Recover.html).

Acestea sunt frecvențele cunoscute ale textului în clar:

In [ ]:
FREQS = {
    "A": 0.07048643054277828,
    "C": 0.01577161913523459,
    "B": 0.012074517019319227,
    "E": 0.13185372585096597,
    "D": 0.043393514259429625,
    "G": 0.01952621895124195,
    "F": 0.023867295308187673,
    "I": 0.06153403863845446,
    "H": 0.08655128794848206,
    "K": 0.007566697332106716,
    "J": 0.0017594296228150873,
    "M": 0.029657313707451703,
    "L": 0.04609015639374425,
    "O": 0.07679967801287949,
    "N": 0.060217341306347746,
    "Q": 0.0006382244710211592,
    "P": 0.014357175712971482,
    "S": 0.05892939282428703,
    "R": 0.05765294388224471,
    "U": 0.02749540018399264,
    "T": 0.09984475620975161,
    "W": 0.01892824287028519,
    "V": 0.011148804047838086,
    "Y": 0.023045078196872126,
    "X": 0.0005289788408463661,
    "Z": 0.00028173873045078196,
}

In [ ]:
def compute_distribution(frequencies: Dict[str, float]) -> float:
    """Computes the chi-distribution based on a dictionary of frequencies
    relative to the FREQS frequencies dictionary."""
    chi_2 = 0.0
    for l in FREQS:
        chi_2 += (frequencies[l] - FREQS[l]) ** 2 / FREQS[l]
    return chi_2


def split_in_cosets(text: str, keylen: int) -> List[List[str]]:
    """Splits a text in keylen cosets."""
    cosets = []
    for i in range(keylen):
        coset = []
        for j in range(i, len(text), keylen):
            coset.append(text[j])
        cosets.append(coset)
    return cosets


def merge_cosets(cosets: List[List[str]], coset_size: int) -> str:
    """Merges the cosets to obtain the original text."""
    text = ""
    for j in range(coset_size):
        for i in range(len(cosets)):
            text += cosets[i][j]
    return text

In [ ]:
def get_freq_dict(coset: List[str], shift: int) -> Dict[str, float]:
    """Computes the frequency table for a coset shifted to left with a
    given shift."""
    freq = {}

    # TODO 1 compute the frequency of the letters in the coset shifted to left
    # by the shift parameter

    return freq

In [ ]:
def find_correct_shift(coset: List[str]) -> int:
    """Returns the shift computed for a coset."""
    shift = 0

    # TODO 2 compute the shift which leads to the lowest chi-distribution

    return shift

In [ ]:
with open("msg_ex3.txt", mode="r", encoding="utf-8") as myfile:
    text = myfile.read().strip()

dec_text = ""

# TODO 3 decrypt the text

print(dec_text)

## Bonus: Exercițiul 4 (2p)

 La curs am spus că One Time Pad este maleabil (i.e. putem schimba cu ușurință criptarea unui text în clar prin simpla schimbare a ciphertextului). De asemenea, am discutat cum CRC este o idee proastă de design pentru protocolul WEP, din cauza proprietății de linearitate.

Vi se dă următorul ciphertext, în hexazecimal:
``` text
021e0e061d1694c9
```

care corespunde concatenării mesajelor “floare” și CRC-16 asociat mesajului (în hexa este “8E31”) obținut folosind acest website: http://www.lammertbies.nl/comm/info/crc-calculation.html

Dacă trebuie să modificăm ciphertextul astfel încât o decriptare corectă să ducă la “albina” în loc de “floare”, iar calcularea CRC-16 să rămână corectă, care este modificarea pe care trebuie să o realizăm?

Afișați noul ciphertext după modificările necesare, și arătați că în mod corect duce la plaintextul “albina” și un CRC-16 calculat corect.

Poate găsiți acest început de script util:

In [ ]:
# Plaintexts
s1 = "floare"
s2 = "albina"
G = ""  # To find

# Obtain crc of s1
# See this site:
# http://www.lammertbies.nl/comm/info/crc-calculation.html
x1 = str_2_hex(s1)
x2 = str_2_hex(s2)
print("x1: " + x1)
crc1 = "8E31"  # CRC-16 of x1

# Compute delta (xor) of x1 and x2:
xd = hexxor(x1, x2)
print("xd: " + xd)

# TODO:


Folosiți proprietatea pentru CRC-16 cum că CRC(m XOR d) = CRC(m) XOR CRC(d).

Dacă d = 'floare' XOR 'albina', iar C = [C1 | C2] = [m XOR G1 | CRC(m) XOR G2], atunci C1' = C1 XOR d.